# LegalBERT for Next Word Prediction

## Install/Import Libraries

In [ ]:
# pip install transformers datasets torch
# pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import nltk
from nltk.tokenize import sent_tokenize
import re
import random
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from datasets import load_from_disk, Dataset
import os
import numpy as np


## Load and Preprocess Dataset (US Bills)

In [ ]:
# Load the 'us_bills' subset

dataset = load_dataset("pile-of-law/pile-of-law", "us_bills")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/25.6k [00:00<?, ?B/s]

pile-of-law.py:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

The repository for pile-of-law/pile-of-law contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/pile-of-law/pile-of-law.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Generating dataset pile-of-law (/root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)
INFO:datasets.builder:Generating dataset pile-of-law (/root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)
INFO:datasets.builder:Downloading and preparing dataset pile-of-law/us_bills to /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60...


...

INFO:datasets.download.download_manager:Downloading took 0.0 min
Checksum Computation took 0.0 min
INFO:datasets.download.download_manager:Checksum Computation took 0.0 min
Generating train split
INFO:datasets.builder:Generating train split


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split
INFO:datasets.builder:Generating validation split


Generating validation split: 0 examples [00:00, ? examples/s]

Unable to verify splits sizes.
INFO:datasets.utils.info_utils:Unable to verify splits sizes.
Dataset pile-of-law downloaded and prepared to /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60. Subsequent calls will reuse this data.
INFO:datasets.builder:Dataset pile-of-law downloaded and prepared to /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60. Subsequent calls will reuse this data.


In [ ]:
def clean_text(bills):
    clean_bills = []
    for bill in bills:
      # Remove all newline and tab characters
      text = bill.replace('\n', ' ').replace('\t', ' ')
      # Remove sequences of exactly three lowercase letters
      text = re.sub(r'\[([a-z]{1,3})\]', ' ', text, flags=re.IGNORECASE)
      clean_bills.append(text.lower())

    return clean_bills

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
#sentences = sent_tokenize(clean_text(dataset['train']['text'][:2]))
#print(sentences[:5])
clean_trained = clean_text(dataset['train']['text'][:60])
clean_trained_sent = []
for bill in clean_trained:
  sent = sent_tokenize(bill)
  clean_trained_sent.extend(sent)

clean_val = clean_text(dataset['validation']['text'][120:130])
clean_val_sent = []
for bill in clean_val:
  sent = sent_tokenize(bill)
  clean_val_sent.extend(sent)

print(len(clean_val_sent))

3126


## Next Word Prediction

Masking

In [ ]:
# Step 1: Load the data
train_dataset = clean_trained_sent
test_dataset = clean_val_sent #load_from_disk('/content/drive/MyDrive/CS 7650/preprocessed_us_bills/validation')

def tokenize_and_mask_last_word(examples):
    inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128, return_tensors="pt")

    # Clone input_ids to create labels
    inputs['labels'] = inputs['input_ids'].clone()

    # Mask the last word in each sentence
    for i in range(len(inputs['input_ids'])):
        # Find the index of the last non-padding token
        non_pad_tokens = (inputs['input_ids'][i] != tokenizer.pad_token_id).nonzero(as_tuple=True)[0]
        if len(non_pad_tokens) > 1:
            last_token_idx = non_pad_tokens[-1].item()
            inputs['input_ids'][i][last_token_idx] = tokenizer.mask_token_id  # Mask the last word
    return inputs


# Step 2: Randomly sample 1000 examples from the train dataset and 200 from the test dataset
train_sample = Dataset.from_dict({'text': train_dataset})#.shuffle(seed=42).select(range(1000))
test_sample = Dataset.from_dict({'text': test_dataset})#.shuffle(seed=42).select(range(200))

# Step 3: Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = BertForMaskedLM.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Function to tokenize inputs with mask token for next-word prediction
# def tokenize_function(examples):
#     # Use the DataCollatorForLanguageModeling to automatically mask tokens
#     #return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
#     inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
#     inputs['labels'] = inputs['input_ids'].clone()  # Copy input_ids as labels
#     inputs['labels'][:, :-1] = inputs['labels'][:, 1:]  # Shift labels one position to the right
#     inputs['labels'][:, -1] = -100  # Pad the last position with -100 to ignore
#     return inputs

# Apply tokenization
train_sample = train_sample.map(tokenize_and_mask_last_word, batched=True)
test_sample = test_sample.map(tokenize_and_mask_last_word, batched=True)



Map:   0%|          | 0/1311 [00:00<?, ? examples/s]

Map:   0%|          | 0/3126 [00:00<?, ? examples/s]

Evaluation

In [ ]:
# Step 6: Calculate accuracy on test data for both fine-tuned and pretrained models
def compute_accuracy(model, dataset):
    model.eval()
    correct, total = 0, 0
    # wrong_predictions = []  # Store sentences with wrong predictions
    # right_predictions = []  # Store sentences with right predictions
    # max_examples = 5 # Maximum number of examples to store for each case
    total_loss = 0
    total_tokens = 0
    device = torch.device('cpu')
    model.to(device)
    with torch.no_grad():
        for example in dataset:
            # Convert list values to tensors
            inputs = {
                k: torch.tensor(v).unsqueeze(0).to(device) if isinstance(v, list) else v.unsqueeze(0).to(device) #check if v is list, else unsqueeze
                for k, v in example.items()
                if k in ["input_ids", "attention_mask"]
            }
            labels = inputs["input_ids"].clone()  # Copy the input_ids as labels for next word prediction
            # Replace masked token IDs in labels with -100 to ignore them in accuracy calculation
            labels[labels == tokenizer.mask_token_id] = -100
            outputs = model(**inputs, labels=labels)
            predictions = outputs.logits.argmax(dim=-1) #Try getting
            # for i in range(len(predictions[0])):
            #     if labels[0][i].item() != -100:  # Ignore masked tokens
            #         predicted_token_id = predictions[0][i].item()
            #         actual_token_id = labels[0][i].item()

            #         if predicted_token_id != actual_token_id:
            #             # Get the toriginal sentence (before tokenization)
            #             original_sentence = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
            #             # Get the predicted and actual words
            #             predicted_word = tokenizer.decode([predicted_token_id])
            #             actual_word = tokenizer.decode([actual_token_id])
            #             # Store the sentence and prediction information
            #             if len(wrong_predictions) < max_examples:
            #                 wrong_predictions.append((original_sentence, predicted_word, actual_word))
            #         else:
            #             # Get the original sentence (before tokenization)
            #             original_sentence = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
            #             # Get the predicted and actual words
            #             predicted_word = tokenizer.decode([predicted_token_id])
            #             actual_word = tokenizer.decode([actual_token_id])
            #             # Store the sentence and prediction information
            #             if len(right_predictions) < max_examples:
            #                 right_predictions.append((original_sentence, predicted_word, actual_word))
            # decoded_labels = [tokenizer.decode([token_id], skip_special_tokens=True) for token_id in labels[labels != -100]]
            # decoded_predictions = [tokenizer.decode([token_id], skip_special_tokens=True) for token_id in predictions[labels != -100]]
            # # Only compare predictions and labels where labels are not -100
            # print("Sample decoded labels:", decoded_labels)
            # print("Sample decoded predictions:", decoded_predictions)

            loss = outputs.loss
            num_tokens = (labels != -100).sum().item()
            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens

            correct += (predictions[0][labels[0] != -100] == labels[0][labels[0] != -100]).sum().item()
            total += (labels[0] != -100).sum().item()

    # print("\nExamples of Wrong Predictions:")
    # for sentence, predicted_word, actual_word in wrong_predictions:
    #     print(f"Sentence: {sentence}")
    #     print(f"Predicted: {predicted_word}, Actual: {actual_word}\n")

    # print("\nExamples of Right Predictions:")
    # for sentence, predicted_word, actual_word in right_predictions:
    #     print(f"Sentence: {sentence}")
    #     print(f"Predicted: {predicted_word}, Actual: {actual_word}\n")
    perplexity = np.exp(total_loss / total_tokens)
    return correct / total, perplexity

# Accuracy with fine-tuned model
fine_tuned_accuracy, fine_tuned_perplexity = compute_accuracy(model, test_sample)
print("Fine-tuned Model Accuracy:", fine_tuned_accuracy)
print("Fine-tuned Model Perplexity:", fine_tuned_perplexity)


Fine-tuned Model Accuracy: 0.3828570133147944
Fine-tuned Model Perplexity: 12214.637011078212


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 